In [11]:
import json
from collections import Counter
import regex as re
import os
# from transformers import AutoTokenizer #4.25.1 torch 1.7.1
import pyphen
import pandas as pd
from tokenizers import Tokenizer
from sklearn.preprocessing import MinMaxScaler
import random
from tokenizers import ByteLevelBPETokenizer
from tokenizers.pre_tokenizers import ByteLevel

In [ ]:
folder_to_process='strict_model_ordered_data_reverse'
path='./SlovakBabyLM/Curricullum_learning/data/'

### CREATE WHOLE DATA JSON

Create whole json file from folders

In [ ]:
full_json_text=[]
for subdir, dirs, files in os.walk(f'{path}{folder_to_process}'):
    for dir in dirs: 
        folder_path = os.path.join(subdir, dir)
        source=0
        for files in os.listdir(folder_path):
            file_path = os.path.join(folder_path, files)
            if '_processed' not in file_path and '_reordered' not in file_path :
                print(file_path)
                with open(file_path,encoding='utf-8') as data_file: 
                    data = json.load(data_file)
                    full_json_text.extend(data)

/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/fairytales_full/fairytales_full.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/edu/edu.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/childes/childes.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/wiki_full/wiki_full.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/subs/subs.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/lit/lit.json


Create whole json file from folder

In [ ]:
full_json_text=[]
for subdir, dirs, files in os.walk(f'{data}/wiki'):
    for file in files: 
        file_path = os.path.join(subdir, file)
        source=0
        print(file_path)
        with open(file_path,encoding='utf-8') as data_file: 
            data = json.load(data_file)
            full_json_text.extend(data)

In [ ]:
with open(f'{data}{folder_to_process}_full_data/{folder_to_process}_full_data_processed.json','a',encoding="utf8") as f_reorded:
    json.dump(full_json_text, f_reorded, ensure_ascii=False, indent=4)

Reshuffle of JSon

In [ ]:
full_text=''
full_json_text=random.sample(full_json_text,len(full_json_text))
for i in full_json_text:
    full_text += f"{i['page'] }"
with open(f'{path}{folder_to_process}_results/{folder_to_process}_randomize.txt','a',encoding="utf8") as f:
    f.write(full_text)

Delete evaluation

In [ ]:
with open(f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data_processed.json','r',encoding="utf8") as f:
    data = json.load(f)

cleaned_data = [{"url": item["url"], "page": item["page"]} for item in data]

with open(f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data_processed.json', "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, ensure_ascii=False, indent=4)

Delete index

In [ ]:
path=f'{path}{folder_to_process}'

for subdir, dirs, files in os.walk(path):
    for dir in dirs: 
        folder_path = os.path.join(subdir, dir)
        for files in os.listdir(folder_path):
            file_path = os.path.join(folder_path, files)
            print(file_path)
            with open(file_path,'r',encoding="utf8") as f:
                data = json.load(f)
                for index, dictionaries in enumerate(data):
                    dictionaries['index']= index
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/fairytales_full/fairytales_full.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/edu/edu.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/childes/childes.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/wiki_full/wiki_full.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/subs/subs.json
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/lit/lit.json


In [ ]:
with open(f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data_processed.json','r',encoding="utf8") as f:
    data = json.load(f)

for index, dictionaries in enumerate(data):
    dictionaries['index']= index

with open(f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data_processed.json', "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

### Train tokenizer

In [ ]:
import os
import json

load_json_path=f'{path}{folder_to_process}'
save_path=f'/data/lubosk/diploma_thesis/tok_bpe/{folder_to_process}_BPE/text_tokenizer.txt'
tokenizer_dir = f"/data/lubosk/diploma_thesis/tok_bpe/{folder_to_process}_BPE"

os.makedirs(tokenizer_dir, exist_ok=True)


def define_txt_file(load_path,save_path):
    text=''
    for subdir, dirs, files in os.walk(load_path):
        for dir in dirs: 
            folder_path = os.path.join(subdir, dir)
            for files in os.listdir(folder_path):
                file_path = os.path.join(folder_path, files)
                with open(file_path, "r", encoding="utf8") as f:
                    if not "_processed" in file_path and not "_reordered" in file_path:
                        print(f'\n Start Process: {file_path}')
                        data = json.load(f)
                        for i in data:
                            if isinstance(i['page'],list):
                                i['page']=' '.join(i['page'])
                            text+=i['page']
    with open(save_path, "w", encoding="utf-8") as f:
        f.write(text)


tokenizer = ByteLevelBPETokenizer()
tokenizer.pre_tokenizer = ByteLevel()


define_txt_file(load_json_path,save_path)

tokenizer.train(
    [save_path],
    vocab_size=60000, 
    min_frequency=2, 
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    show_progress=True
)

tokenizer_json_path = os.path.join(tokenizer_dir, "tokenizer.json")
tokenizer.save(tokenizer_json_path)



 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/fairytales_full/fairytales_full.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/edu/edu.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/childes/childes.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/wiki_full/wiki_full.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/subs/subs.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/lit/lit.json





# Choose randomly from files

In [22]:
import os 
import json
import random
import regex as re
folder_to_process='evaluation_data'
condition_files='_reordered'

In [17]:
def preprocess_text_f(text):
    text=re.sub(r'([.,!?;:\'\"])\n+\1', r'\1',text)
    text=re.sub(r'([.,!?;:\'\"])\s+\1', r'\1', text)
    text = re.sub(r'([!?;])\1+', r'\1', text)  
    text=re.sub(r' +',r' ',text)
    text=re.sub(r'(?<=\n)\s*\d+\s*(?=\n)',r'\n',text)
    return text

In [ ]:
# where is data taken and how many 
paths=[
        {'size_words':470_000,'path':f'{path}childes'},
        {'size_words':2_300_000,'path':f'{path}wiki_full'},
        {'size_words':910_000,'path':f'{path}fairytales_full'},
        {'size_words':4_000_000,'path':f'{path}subs'},
        {'size_words':1_304_000,'path':f'{path}edu'},
        {'size_words':990_000,'path':f'{path}lit'},
       ]
for path in paths:
    for subdir, dirs, files in os.walk(path['path']):
        files=[item for item in files if '_processed' not in item and '_reordered' not in item]
        counter= path['size_words']/len(files)
        json_list=[]
        additional=0
        for file in files: 
            file_path = os.path.join(subdir, file)
            print(file_path)
            with open(file_path, "r", encoding="utf8") as f:
                data = json.load(f)
            length_data=0
            selected_indices = set(range(0, len(data)))
            try: 
                while selected_indices and length_data <= counter + additional:
                    random_number = random.choice(tuple(selected_indices))
                    selected_indices.remove(random_number)
                    data[random_number]['page']=preprocess_text_f(data[random_number]['page'])
                    json_list.append(data[random_number])
                    length_data=length_data+len(data[random_number]['page'].split(' '))
                if len(selected_indices) ==0:
                    parts = subdir.split('/')
                    print(f'not enough text for size_words only {length_data} for {parts[-1]}:{file}')
                    additional= counter-length_data +additional
                else: 
                    additional=0
                    print(f'additional: {additional}')
            except Exception as e:
                print(e)
        parts = subdir.split('/')
        parts[-2] =  folder_to_process
        new_path = '/'.join(parts)
        with open(f'{new_path}/{parts[-1]}.json', "a", encoding="utf-8") as f:
            json.dump(json_list, f, ensure_ascii=False, indent=4)

/data/lubosk/diploma_thesis/data/subs/json_subtitles.json
additional: 0


# Choose by order

In [34]:
import os 
import json
import random
folder_to_process='strict_model_ordered_data_reverse'
condition_files='_reordered'

In [ ]:
paths=[
        {'size_words':470_000,'path':f'{path}childes'},
        {'size_words':2_300_000,'path':f'{path}wiki_full'},
        {'size_words':910_000,'path':f'{path}fairytales_full'},
        {'size_words':4_000_000,'path':f'{path}subs'},
        {'size_words':1_304_000,'path':f'{path}edu'},
        {'size_words':990_000,'path':f'{path}lit'},
       ]
for path in paths:
    for subdir, dirs, files in os.walk(path['path']):
        for file in files:
            if condition_files in file:  
                file_path = os.path.join(subdir, file)
                json_list=[]
                length_data=0
                print(file_path)
                with open(file_path, "r", encoding="utf8") as f:
                    data = json.load(f)
                data.reverse()
                for source in data:
                    if length_data <= path['size_words']:
                        length_data+= len(source['page'].split())
                        json_list.append(source)
                parts = subdir.split('/')
                parts[-2] =  folder_to_process
                new_path = '/'.join(parts)
                with open(f'{new_path}/{parts[-1]}.json', "a", encoding="utf-8") as f:
                    json.dump(json_list, f, ensure_ascii=False, indent=4)

/data/lubosk/diploma_thesis/data/childes/final_conv_reordered_sum_freq_gramma.json
/data/lubosk/diploma_thesis/data/wiki_full/wiki_full_data_reordered_sum_freq_gramma.json
/data/lubosk/diploma_thesis/data/fairytales_full/fairytales_full_data_reordered_sum_freq_gramma.json
/data/lubosk/diploma_thesis/data/subs/json_subtitles_reordered_sum_freq_gramma.json
/data/lubosk/diploma_thesis/data/edu/tahaky_referaty-prep_reordered_sum_freq_gramma.json
/data/lubosk/diploma_thesis/data/lit/test_json_files_new_proc_reordered_sum_freq_gramma.json


Counter of words

In [ ]:
a=0
for source in data:
    a+=len(source['page'].split())

'strict_model_ordered_data'

In [1]:
import json
import os 

In [ ]:
total_word_count=0
for subdir, dirs, files in os.walk(f'{path}'):
    for dir in dirs:
        folder_path = os.path.join(subdir, dir)
        source=0
        for files in os.listdir(folder_path):
            file_path = os.path.join(folder_path, files)
            with open(file_path,encoding='utf-8') as data_file:    
                data = json.load(data_file)
                for i in data:
                    source= source+len(i['page'].split())
                print(file_path)
                print("{:,}".format(source))
                print(len(data))
                total_word_count=total_word_count+source
                source=0
print(f'Total: {"{:,}".format(total_word_count)}')

/data/lubosk/diploma_thesis/data/fairytales_full/fairytales_full_data.json
5,112,961
6895
/data/lubosk/diploma_thesis/data/fairytales_full/fairytales_full_data_processed.json
5,112,961
6895
/data/lubosk/diploma_thesis/data/fairytales_full/fairytales_full_data_reordered_sum_freq_gramma.json
5,112,961
6895
/data/lubosk/diploma_thesis/data/fairytales/zlatyfond_preprocessed.json
671,388
293
/data/lubosk/diploma_thesis/data/fairytales/PDF_fairytales.json
509,365
30
/data/lubosk/diploma_thesis/data/fairytales/created_fairytales.json
1,786,974
3094
/data/lubosk/diploma_thesis/data/fairytales/rozpravky_online_preprocessed.json
43,636
87
/data/lubosk/diploma_thesis/data/fairytales/svetrozpravok_preprocessed.json
38,773
70
/data/lubosk/diploma_thesis/data/fairytales/readmio.json
358,381
1537
/data/lubosk/diploma_thesis/data/fairytales/sikovnamamina_preprocessed.json
41,392
36
/data/lubosk/diploma_thesis/data/fairytales/zones_preprocessed.json
1,359,908
1058
/data/lubosk/diploma_thesis/data/fairy